In [1]:
import cv2
import dlib
import numpy as np

PREDICTOR_PATH = "F:/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor()

def get_landmarks(image):
    rects = detector(image)
    if len(rects)>1:
        return "Error"
    if len(rects)==0:
        return "Error"
    return np.matrix([[p.x,p.y] for p in predictor(image,rects[0]).parts()])

def annotate_landmarks(im,landmarks):
    im = im.copy()
    for id,points in enumerate(landmarks):
        points = (points[0,0],points[0,1])
        cv2.putText(im,str(id),points,cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,0.4,color=(0.0,255))
        cv2.circle(im,points,3,color=(0,255,255))
    return im

def top_lip(landmarks):
    top_lip_pts = []
    for i in range(50,53):
        top_lip_pts.append(landmarks[i])
    for i in range(61,64):
        top_lip_pts.append(landmarks[i])
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    top_lip_mean = np.mean(top_lip_pts,axis=0)
    return int(top_lip_mean[:,1])

def bottom_lip(landmarks):
    bottom_lip_pts = []
    for i in range(65,68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56,59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts,axis=0)
    return int(bottom_lip_mean[:,1])

def mouth_open(image):
    landmarks = get_landmarks(image)
    if landmarks == "Error":
        return image,0
    image_with_landmarks = annotate_landmarks(image,landmarks)
    top_lip_center = top_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(top_lip_center-bottom_lip_center)
    return  image_with_landmarks,lip_distance

cap = cv2.VideoCapture(0)
yawns = 0
yawn_status = False
while True:
    ret,frame = cap.read()
    image_landmarks, lip_distance = mouth_open(frame)
    prev_yawn_status = yawn_status
    if lip_distance > 15:
        yawn_status = True

        cv2.putText(frame,"Subject is Yawing",(50,450),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,127),2)
    else:
        yawn_status = False

    if prev_yawn_status == True and yawn_status == False:
        yawns +=1

    cv2.imshow("Live Landmarks",image_landmarks)
    cv2.imshow('Yawn Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: dlib.fhog_object_detector, image: array, upsample_num_times: int=0) -> dlib.rectangles

Invoked with: <dlib.fhog_object_detector object at 0x000001E71CA27C70>, None